In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

1. # Step 1: Importing and understanding data

In [ ]:
#Import /kaggle/input/covid19-global-forecasting-week-4/train.csv
corona_train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/train.csv')
#Import /kaggle/input/covid19-global-forecasting-week-4/submission.csv

#Import /kaggle/input/covid19-global-forecasting-week-4/test.csv
corona_test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/test.csv')

# Step 2: Clean & Format

In [ ]:
# print(corona_train.head())
# print(corona_train.tail())
# print(corona_test.head())
# print(corona_test.tail())

x_train = corona_train

x_train['Province_State'] = x_train['Province_State'].fillna('')
x_train['Location'] = x_train['Country_Region'] + '_' + x_train['Province_State']

from datetime import datetime
x_train['Date'] = pd.to_datetime(x_train['Date'])

START_DATE = x_train['Date'].min()

x_train['Days'] = x_train['Date'].subtract(START_DATE)
x_train['Days'] = x_train['Days'].map(lambda tdelta_obj: tdelta_obj.days)
# one_hot_encoded_locations = pd.get_dummies(x_train['Location'], drop_first=True)
# x_train = pd.concat([x_train, one_hot_encoded_locations], axis=1)

# x_train_final['Days'] = days / (days.max() - days.min())

x_train

# Step 3: Filter & group data

In [ ]:
# nonzero
x_train_c = x_train.loc[x_train['ConfirmedCases'] > 0]
x_train_f = x_train.loc[x_train['Fatalities'] > 0]
x_train_f

x_c_map = {}
x_f_map = {}

locations = set(x_train.Location.values)
locations

for location in locations:
    x_c_map[location] = x_train_c.loc[x_train_c['Location'] == location]
    x_f_map[location] = x_train_f.loc[x_train_f['Location'] == location]

# Step 4: Test with a single location

In [ ]:
def predict_train(location):
    x_train_final = x_c_map[location] \
    .drop(['Id', 'Province_State', 'Country_Region', 'Date', 'ConfirmedCases', 'Fatalities', 'Location'], axis=1)
    y_train_final_c = x_c_map[location] \
    .drop(['Id', 'Province_State', 'Country_Region', 'Date', 'Location', 'Days', 'Fatalities'], axis=1)
    y_train_final_f = x_c_map[location] \
    .drop(['Id', 'Province_State', 'Country_Region', 'Date', 'Location', 'Days', 'ConfirmedCases'], axis=1)
    
    # fit to quadratic
    y_train_final_c_input = np.power(y_train_final_c, 0.5)
    y_train_final_f_input = np.power(y_train_final_f, 0.5)

    from sklearn.linear_model import LinearRegression
    lm = LinearRegression()
    lm.fit(x_train_final, y_train_final_c_input)
    # print the intercept
    print(lm.intercept_)

    y_pred = lm.predict(x_train_final)
    y_pred = np.power(y_pred, 2)

    #Actual vs Predicted
    import matplotlib.pyplot as plt
    c = [i for i in range(1,len(y_pred) + 1,1)]
    fig = plt.figure()
    plt.plot(c,y_train_final_c, color="blue", linewidth=2.5, linestyle="-")
    plt.plot(c,y_pred, color="red",  linewidth=2.5, linestyle="-")
    fig.suptitle('Actual and Predicted', fontsize=20)              # Plot heading 
    plt.xlabel('Location and Date', fontsize=18)                               # X-label
    plt.ylabel('Confirmed Cases', fontsize=16)

    from sklearn.metrics import mean_squared_error, r2_score
    mse = mean_squared_error(y_train_final_c, y_pred)
    r_squared = r2_score(y_train_final_c, y_pred)
    print('Mean_Squared_Error :' ,mse)
    print('r_square_value :',r_squared)
    
    lm = LinearRegression()
    lm.fit(x_train_final, y_train_final_f_input)
    # print the intercept
    print(lm.intercept_)

    y_pred = lm.predict(x_train_final)
    y_pred = np.power(y_pred, 2)

    #Actual vs Predicted
    import matplotlib.pyplot as plt
    c = [i for i in range(1,len(y_pred) + 1,1)]
    fig = plt.figure()
    plt.plot(c,y_train_final_f, color="blue", linewidth=2.5, linestyle="-")
    plt.plot(c,y_pred, color="red",  linewidth=2.5, linestyle="-")
    fig.suptitle('Actual and Predicted', fontsize=20)              # Plot heading 
    plt.xlabel('Location and Date', fontsize=18)                               # X-label
    plt.ylabel('Fatalities', fontsize=16)

    from sklearn.metrics import mean_squared_error, r2_score
    mse = mean_squared_error(y_train_final_f, y_pred)
    r_squared = r2_score(y_train_final_f, y_pred)
    print('Mean_Squared_Error :' ,mse)
    print('r_square_value :',r_squared)

predict_train('US_California')

# Step 5: Prepare test data

In [ ]:
x_test = corona_test

x_test['Province_State'] = x_test['Province_State'].fillna('')
x_test['Location'] = x_test['Country_Region'] + '_' + x_test['Province_State']

from datetime import datetime
x_test['Date'] = pd.to_datetime(x_test['Date'])

START_DATE = x_train['Date'].min()

x_test['Days'] = x_test['Date'].subtract(START_DATE)
x_test['Days'] = x_test['Days'].map(lambda tdelta_obj: tdelta_obj.days)
# one_hot_encoded_locations = pd.get_dummies(x_train['Location'], drop_first=True)
# x_train = pd.concat([x_train, one_hot_encoded_locations], axis=1)

# x_train_final['Days'] = days / (days.max() - days.min())

x_test

In [ ]:
x_test['ConfirmedCases'] = -1
x_test['Fatalities'] = -1
def predict(location, graph=False):
    x_train_final = x_c_map[location] \
    .drop(['Id', 'Province_State', 'Country_Region', 'Date', 'ConfirmedCases', 'Fatalities', 'Location'], axis=1)
    y_train_final_c = x_c_map[location] \
    .drop(['Id', 'Province_State', 'Country_Region', 'Date', 'Location', 'Days', 'Fatalities'], axis=1)
    y_train_final_f = x_c_map[location] \
    .drop(['Id', 'Province_State', 'Country_Region', 'Date', 'Location', 'Days', 'ConfirmedCases'], axis=1)
    
    x_test_final = x_test.loc[x_test['Location'] == location] \
    .drop(['ForecastId', 'Province_State', 'Country_Region', 'Date', 'Location', 'ConfirmedCases', 'Fatalities'], axis=1)
    
    # fit to quadratic
    y_train_final_c_input = np.power(y_train_final_c, 0.5)
    y_train_final_f_input = np.power(y_train_final_f, 0.5)

    from sklearn.linear_model import LinearRegression
    lm = LinearRegression()
    lm.fit(x_train_final, y_train_final_c_input)

    y_pred_c = lm.predict(x_test_final)
    y_pred_c = np.power(y_pred_c, 2)

    if graph:
        # print the intercept
        print(lm.intercept_)
        #Predicted
        import matplotlib.pyplot as plt
        c = [i for i in range(1,len(y_pred_c) + 1,1)]
        fig = plt.figure()
        plt.plot(c,y_pred_c, color="red",  linewidth=2.5, linestyle="-")
        fig.suptitle('Predicted', fontsize=20)              # Plot heading 
        plt.xlabel('Location and Date', fontsize=18)                               # X-label
        plt.ylabel('Confirmed Cases', fontsize=16)
    
    lm = LinearRegression()
    lm.fit(x_train_final, y_train_final_f_input)

    y_pred_f = lm.predict(x_test_final)
    y_pred_f = np.power(y_pred_f, 2)

    if graph:
        # print the intercept
        print(lm.intercept_)
        #Predicted
        import matplotlib.pyplot as plt
        c = [i for i in range(1,len(y_pred_f) + 1,1)]
        fig = plt.figure()
        plt.plot(c,y_pred_f, color="red",  linewidth=2.5, linestyle="-")
        fig.suptitle('Predicted', fontsize=20)              # Plot heading 
        plt.xlabel('Location and Date', fontsize=18)                               # X-label
        plt.ylabel('Fatalities', fontsize=16)
 
    return y_pred_c, y_pred_f

c, f = predict('US_California', graph=True)

In [ ]:
for location in x_c_map:
    c, f = predict(location)
    x_test.loc[x_test['Location'] == location, 'ConfirmedCases'] = c
    x_test.loc[x_test['Location'] == location, 'Fatalities'] = f

In [ ]:
x_test.drop(['Province_State', 'Country_Region', 'Date', 'Location', 'Days'], axis=1, inplace=True)
x_test.to_csv('submission.csv',index=False)